[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/analytics-and-ml/model-training/gpl/01-query-gen.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/analytics-and-ml/model-training/gpl/01-query-gen.ipynb)

We should have the CORD-19 *document_parses/pdf_json* directory locally, we will be reading text data from these files. First we define a generator to do this.

In [4]:
from pathlib import Path
import json

paths = [str(path) for path in Path('document_parses/pdf_json').glob('*.json')]

def get_text():
    passage_dupes = set()
    for path in paths:
        with open(path, 'r') as fp:
            doc = json.load(fp)
        # extract the passages of text from each document
        body_text = [line['text'] for line in doc['body_text']]
        # loop through and yield one passage at a time
        for passage in body_text:
            # check it is not a duplicate first
            if passage in passage_dupes:
                pass
            else:
                passage_dupes.add(passage)
                yield passage

Let's test the generator...

In [5]:
passages = get_text()

for i, passage in enumerate(passages):
    print(passage)
    if i == 2: break

Until recently, seven types of coronaviruses had been reported to cause infections in humans [1] . Coronaviruses can use animal hosts and then can evolve to infect humans. This process is thought to explain the emergence of SARS-CoV (severe respiratory distress syndrome coronavirus) in 2003, MERS-CoV (Middle Eastern Respiratory Syndrome) in 2012, and SARS-CoV-2 in 2019. SARS-CoV-2 has 82% genome sequence similarity to SARS-CoV and 50% genome sequence homology to MERS-CoV. COVID-19 symptoms range from mild (fever, cough, or dyspnea) to moderate (respiratory failure requiring oxygen support) and can progress to ARDS (acute respiratory distress syndrome) and multiorgan failure. In one of the earlier studies, 80% of cases were mild, but the mortality rate ranged from 1.86% to 9.86% [2] . Higher mortality rates were reported in countries like Italy, possibly secondary to resource depletion in an overwhelmed health care system. Gastrointestinal symptoms such as diarrhea have been reported in

Perfect, we have our passage generator function, now we need to create queries to pair with these passages. We do this using a T5 query generation model called `'doc2query/msmarco-t5-base-v1'`, which was trained on a pre-COVID dataset form 2018. The model still produces sensible COVID related queries by copying words from the input text.

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = 'doc2query/msmarco-t5-base-v1'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()

Let's test it on an example.

In [7]:
for passage in passages:
    break

# tokenize the passage
inputs = tokenizer(passage, return_tensors='pt')
# generate three queries
outputs = model.generate(
    input_ids=inputs['input_ids'].cuda(),
    attention_mask=inputs['attention_mask'].cuda(),
    max_length=64,
    do_sample=True,
    top_p=0.95,
    num_return_sequences=3
)

In [8]:
print("Paragraph:")
print(passage)

print("\nGenerated Queries:")
for i in range(len(outputs)):
    query = tokenizer.decode(outputs[i], skip_special_tokens=True)
    print(f'{i + 1}: {query}')

Paragraph:
1) It is postulated that both SARS-CoV-2 and SARS-CoV bind to angiotensin-converting enzyme 2 (ACE2) receptors to enter the target cell [14] where the virus replication begins and starts to infect cells of the upper respiratory tract. Based on the scRNA-seq data, Chai et al. [15] found that ACE 2 receptors also found in the hepatobiliary system (high in bile duct cells, cholangiocytes, when compared to liver cells). Cholangiocytes play a critical role in liver regeneration and immune responses [16] . Thus, the authors concluded that potential damage of cholangiocytes by 2019-nCoV might lead to profound consequences in the liver rather than the direct effect of the virus on hepatocytes.

Generated Queries:
1: where is sars infection found
2: sars bacteria affects what organ
3: what are the ace receptors that are associated with sars virus


Looks great, we're getting what mostly look like intelligent queries that the passage answers. We will apply this at a larger scale, creating a large number *(query, passage)* pairs, which we will save to TSV files in a new *data/qp_pairs* directory.

In [9]:
import os

if not os.path.exists('data'): os.mkdir('data')

We will restrict the number of pairs we create, as CORD-19 is a *massive* dataset. Change this threshold as you prefer.

In [10]:
from tqdm.auto import tqdm  # this is our progress bar

target = 200_000
batch_size = 256
num_queries = 3  # number of queries to generate for each passage
count = 0
lines = []
passage_batch = []

# reinitialize passage generator
passages = get_text()

with tqdm(total=target) as progress:
    for passage in passages:
        if count >= target: break
        # remove tab + newline characters if present
        passage_batch.append(passage.replace('\t', ' ').replace('\n', ' '))
        
        # we encode in batches
        if len(passage_batch) == batch_size:
            # tokenize the passage
            inputs = tokenizer(
                passage_batch,
                truncation=True,
                padding=True,
                max_length=256,
                return_tensors='pt'
            )

            # generate three queries per doc/passage
            outputs = model.generate(
                input_ids=inputs['input_ids'].cuda(),
                attention_mask=inputs['attention_mask'].cuda(),
                max_length=64,
                do_sample=True,
                top_p=0.95,
                num_return_sequences=num_queries
            )

            # decode query to human readable text
            decoded_output = tokenizer.batch_decode(
                outputs,
                skip_special_tokens=True
            )

            # loop through to pair query and passages
            for i, query in enumerate(decoded_output):
                query = query.replace('\t', ' ').replace('\n', ' ')  # remove newline + tabs
                passage_idx = int(i/num_queries)  # get index of passage to match query
                lines.append(query+'\t'+passage_batch[passage_idx])
                count += 1
            
            passage_batch = []
            progress.update(len(decoded_output))

# write (Q, P+) pairs to file
with open('./data/pairs.tsv', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(lines))

  0%|          | 0/200000 [00:00<?, ?it/s]

We have now generating queries for our passages and can move onto the next step of GPL called *negative mining*.